In [34]:
from z3 import *

In [1]:
from z3 import Solver, Bool, Xor, If, Or, sat
import numpy as np
from functools import reduce

# Stabilizer Lists => Coordinates

In [7]:
def get_stab_coords(grid):
    stabilizers = {}
    num_cols = len(grid[0])  # Number of columns in the grid

    # Populate the initial stabilizers dictionary
    for i, row in enumerate(grid):
        for j, letter in enumerate(row):
            if letter != '.':
                if letter not in stabilizers:
                    stabilizers[letter] = [[]]  # Initialize with a list containing an empty list
                stabilizers[letter][0].append(i * num_cols + j)  # Append to the first list

    # Iterate through each letter and perform the transformation
    for letter, lists in stabilizers.items():
        original_list = lists[0]

        # Precompute row and column indices for the original list
        indices = [(value // num_cols, value % num_cols) for value in original_list]

        # Generate all transformed lists in one go
        transformed_lists = [
            [
                i * num_cols + ((j + n) % num_cols)
                for i, j in indices
            ]
            for n in range(1, num_cols)
        ]

        # Extend the lists with all the new transformed lists
        lists.extend(transformed_lists)

    return stabilizers

In [8]:
def generate_stabilizer_matrix(grid):
    # Step 2: Prepare stabilizer data
    stab_coords = get_stab_coords(grid)
    num_positions = len(grid) * len(grid[0])
    stabilizers = list(stab_coords.keys())

    # Step 3: Initialize matrix
    matrix = []

    # Step 4: Fill the matrix
    for stabilizer in stabilizers:
        for coord_list in stab_coords[stabilizer]:
            row = [0] * num_positions
            for position in coord_list:
                row[position] = 1
            matrix.append(row)

    return matrix

def gf2_add_row(r1, r2):
    # XOR the rows bitwise
    return [a ^ b for a, b in zip(r1, r2)]

def gf2_rref(matrix):
    # Convert matrix into Reduced Row Echelon Form over GF(2)
    # Returns: RREF matrix, pivot positions
    # This is a standard Gaussian elimination but using XOR instead of addition and no division (just swapping and XOR).
    M = [row[:] for row in matrix]  # Copy
    rows = len(M)
    cols = len(M[0]) if rows > 0 else 0
    
    pivot_positions = []
    pivot_row = 0
    for col in range(cols):
        if pivot_row >= rows:
            break
        
        # Find a pivot row for this column
        pivot = -1
        for r in range(pivot_row, rows):
            if M[r][col] == 1:
                pivot = r
                break
        
        if pivot == -1:
            continue  # No pivot in this column
        
        # Swap if pivot row is not pivot_row
        if pivot != pivot_row:
            M[pivot_row], M[pivot] = M[pivot], M[pivot_row]
        
        # Now eliminate all other 1s in this column
        for r in range(rows):
            if r != pivot_row and M[r][col] == 1:
                M[r] = gf2_add_row(M[r], M[pivot_row])
        
        pivot_positions.append(col)
        pivot_row += 1

    return M, pivot_positions

def convert_stabilizer_matrix_to_generator_matrix(H):
    # Compute RREF of H
    R, pivots = gf2_rref(H)
    n = len(H[0])  # length of each codeword
    r = len(pivots)  # rank of H
    k = n - r
    
    if k <= 0:
        # No code space (trivial code)
        return []
    
    pivot_set = set(pivots)
    free_cols = [c for c in range(n) if c not in pivot_set]

    G = []
    # For each free variable, set that free variable to 1 and the rest 0.
    # Solve for pivot variables using the equations in R.
    for i, free_col in enumerate(free_cols):
        # Start with a candidate vector of length n, all zeros
        vec = [0]*n
        # Set this free variable
        vec[free_col] = 1
        
        # Solve for pivot variables:
        # Since R is in RREF, each pivot row determines one pivot variable
        for row_idx, pcol in enumerate(pivots):
            # The pivot variable at pcol is determined by XORing known values
            # from row R[row_idx]. If we read R[row_idx], the pivot variable equals
            # the XOR of the free variables in that row.
            
            # The equation represented by R[row_idx]:
            # x_pcol = (sum_of_other_variables_in_that_row) since pivot is isolated
            # R[row_idx][pcol] is always 1 because it's a pivot
            # For each non-pivot col that has a 1 in R[row_idx], XOR with vec for that column.
            
            # Determine the right-hand side from row
            rhs = 0
            for c, val in enumerate(R[row_idx]):
                if c != pcol and val == 1:
                    rhs ^= vec[c]
            
            vec[pcol] = rhs
        
        G.append(vec)
    
    return G

# 8 rows, 17 columns => 136 qubits
# 6 unique stabilizers, 17 horizontal translations, 136 qubits => stabilizer will have 6*17 = 102 rows, 136 columns
grid = [
    ['.', '.', '.', '.', '.', 'F', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', 'E', '.', '.', 'F', '.', 'F', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
    ['E', '.', 'E', '.', '.', '.', 'F', '.', '.', '.', '.', '.', '.', 'D', '.', '.', '.'],
    ['E', '.', '.', '.', '.', '.', '.', '.', '.', 'C', '.', '.', '.', '.', 'D', '.', '.'],
    ['.', '.', '.', '.', '.', 'B', '.', '.', '.', '.', 'C', '.', 'D', '.', 'D', '.', '.'],
    ['.', 'A', '.', '.', '.', '.', '.', '.', 'C', '.', 'C', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', 'B', 'B', 'B', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
    ['A', 'A', 'A', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
]

In [9]:
import logging
from functools import reduce
from z3 import Optimize, Bool, Xor, Sum, If, Or, is_true

# Configure logging to output to console and file
logging.basicConfig(
    level=logging.INFO,  # Set to INFO to capture all logging messages
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('hamming_distance.log'),
        logging.StreamHandler()  # Also output to console
    ]
)

def compute_min_hamming_distance(G):
    print("Initializing optimizer and variables.")
    
    # Initialize Z3 optimizer
    optimizer = Optimize()

    # Number of rows in generator matrix G (number of codewords)
    k = len(G)
    n = len(G[0])  # Length of each codeword

    print(f"Generator matrix has {k} rows and {n} columns.")

    # Define Boolean variables for each row of G
    b = [Bool(f'b_{i}') for i in range(k)]

    # Add constraint: At least one row is selected (non-zero codeword)
    optimizer.add(Or(b))
    print("Added constraint: At least one row must be selected.")

    # Helper function for XOR over multiple terms
    def safe_xor(terms):
        if len(terms) == 0:
            return False  # Represents zero in Boolean logic
        elif len(terms) == 1:
            return terms[0]
        else:
            return reduce(lambda x, y: Xor(x, y), terms)

    # Precompute codeword structure
    codeword = []
    for j in range(n):
        terms = [b[i] for i in range(k) if G[i][j] == 1]
        xor_result = safe_xor(terms)
        codeword.append(xor_result)
        logging.debug(f"Computed codeword bit {j}: {xor_result}")

    # Define Hamming weight
    hamming_weight = Sum([If(bit, 1, 0) for bit in codeword])
    print("Defined Hamming weight as the sum of codeword bits.")

    # Ensure non-zero Hamming weight (exclude zero codeword)
    optimizer.add(hamming_weight > 0)

    # Set the objective to minimize the Hamming weight
    optimizer.minimize(hamming_weight)
    print("Set optimization objective to minimize Hamming weight.")

    print("Starting optimization to find minimum non-zero Hamming distance.")
    result = optimizer.check()
    print(f"Optimizer check result: {result}")

    if result == sat:
        model = optimizer.model()
        min_distance = model.evaluate(hamming_weight).as_long()
        selected_rows = [i for i in range(k) if is_true(model.evaluate(b[i]))]
        print(f"Minimum non-zero Hamming distance found: {min_distance}")
        print(f"Selected rows in G contributing to the codeword: {selected_rows}")

        # Compute the codeword values
        computed_codeword = [is_true(model.evaluate(bit)) for bit in codeword]
        binary_codeword = [1 if bit else 0 for bit in computed_codeword]

        # Log the contributing rows
        print("\nContributing rows from G:")
        for idx in selected_rows:
            print(f"Row {idx}: {G[idx]}")

        # Log the resulting codeword
        print("\nXOR Result (codeword):")
        print(f"{binary_codeword}")

        return min_distance
    else:
        print("No valid codeword found.")
        return "No valid codeword found"


In [10]:
H = generate_stabilizer_matrix(grid)
G = convert_stabilizer_matrix_to_generator_matrix(H)
min_distance = compute_min_hamming_distance(G)
print("Minimum distance:", min_distance)

Initializing optimizer and variables.
Generator matrix has 34 rows and 136 columns.
Added constraint: At least one row must be selected.
Defined Hamming weight as the sum of codeword bits.
Set optimization objective to minimize Hamming weight.
Starting optimization to find minimum non-zero Hamming distance.
Optimizer check result: sat
Minimum non-zero Hamming distance found: 22
Selected rows in G contributing to the codeword: [10, 18, 19, 21, 22, 24, 25, 29, 30, 32, 33]

Contributing rows from G:
Row 10: [0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Row 18: [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 

In [23]:
# Generate the stabilizer matrix
G = generate_stabilizer_matrix(grid)

# Compute the minimum Hamming distance
min_distance = compute_min_hamming_distance(G)
print(f"Minimum Hamming distance: {min_distance}")

Initializing optimizer and variables.
Generator matrix has 102 rows and 136 columns.
Added constraint: At least one row must be selected.
Computed codeword bit 0: b_12
Computed codeword bit 1: b_13
Computed codeword bit 2: b_14
Computed codeword bit 3: b_15
Computed codeword bit 4: b_16
Computed codeword bit 5: b_0
Computed codeword bit 6: b_1
Computed codeword bit 7: b_2
Computed codeword bit 8: b_3
Computed codeword bit 9: b_4
Computed codeword bit 10: b_5
Computed codeword bit 11: b_6
Computed codeword bit 12: b_7
Computed codeword bit 13: b_8
Computed codeword bit 14: b_9
Computed codeword bit 15: b_10
Computed codeword bit 16: b_11
Computed codeword bit 17: Xor(Xor(b_11, b_13), b_33)
Computed codeword bit 18: Xor(Xor(b_12, b_14), b_17)
Computed codeword bit 19: Xor(Xor(b_13, b_15), b_18)
Computed codeword bit 20: Xor(Xor(b_14, b_16), b_19)
Computed codeword bit 21: Xor(Xor(b_0, b_15), b_20)
Computed codeword bit 22: Xor(Xor(b_1, b_16), b_21)
Computed codeword bit 23: Xor(Xor(b_0, 

In [6]:
from z3 import Solver, Bool, Xor, Sum, If, Or

# Define the pre-computed minimum Hamming distances for the matrices
precomputed_distances = {
    "G1": 3,  # Pre-computed Hamming distance for G1
    "G2": 2,  # Pre-computed Hamming distance for G2
    "G3": 2,  # Pre-computed Hamming distance for G3
    "G4": 5,  # Pre-computed Hamming distance for G4
    "G5": 2,  # Pre-computed Hamming distance for G5
    "G6": 3,  # Pre-computed Hamming distance for G6
    "G7": 3   # Pre-computed Hamming distance for G7
}

# List of matrices to test
matrices = {
    "G1": [
        [1, 1, 0, 0, 1],
        [0, 1, 1, 1, 0],
        [1, 0, 1, 1, 1]
    ],
    "G2": [
        [1, 0, 1, 0, 1, 1, 0],
        [0, 1, 1, 1, 0, 0, 1],
        [1, 1, 0, 1, 1, 0, 0],
        [0, 0, 1, 1, 0, 1, 1]
    ],
    "G3": [
        [1, 0, 0, 1, 1],
        [0, 1, 0, 1, 0],
        [0, 0, 1, 0, 1],
        [1, 1, 0, 1, 0],
        [0, 1, 1, 0, 1]
    ],
    "G4": [
        [1, 0, 1, 1, 0, 1, 1, 0],
        [0, 1, 0, 1, 1, 0, 1, 1],
        [1, 1, 1, 0, 1, 1, 0, 1]
    ],
    "G5": [
        [1, 0, 1, 1, 0, 0, 1, 0, 1, 1],
        [0, 1, 0, 1, 1, 1, 0, 0, 1, 0],
        [1, 1, 0, 0, 1, 0, 1, 1, 0, 1],
        [0, 0, 1, 1, 0, 1, 0, 1, 1, 0],
        [1, 1, 1, 0, 1, 1, 0, 1, 0, 1],
        [0, 1, 0, 1, 1, 0, 1, 0, 1, 1]
    ],
    "G6": [
        [1, 0, 1, 1, 0, 1],
        [0, 1, 1, 0, 1, 0]
    ],
    "G7": [
        [1, 1, 0, 1, 0, 1, 0, 1, 0],
        [0, 1, 1, 0, 1, 0, 1, 1, 1],
        [1, 0, 1, 1, 1, 0, 0, 1, 1],
        [0, 1, 0, 1, 0, 1, 1, 0, 0]
    ]
}

# Iterate through matrices, compute distances, and verify results
for name, G in matrices.items():
    print(f"Testing {name}...")
    computed_distance = compute_min_hamming_distance(G)
    expected_distance = precomputed_distances[name]

    if computed_distance == expected_distance:
        print(f"PASS: {name} computed distance {computed_distance} matches expected {expected_distance}\n")
    else:
        print(f"FAIL: {name} computed distance {computed_distance} does not match expected {expected_distance}\n")
        
    print("___________________________________________________________________________")



Testing G1...
Initializing optimizer and variables.
Generator matrix has 3 rows and 5 columns.
Added constraint: At least one row must be selected.
Defined Hamming weight as the sum of codeword bits.
Set optimization objective to minimize Hamming weight.
Starting optimization to find minimum non-zero Hamming distance.
Optimizer check result: sat
Minimum non-zero Hamming distance found: 3
Selected rows in G contributing to the codeword: [0]

Contributing rows from G:
Row 0: [1, 1, 0, 0, 1]

XOR Result (codeword):
[1, 1, 0, 0, 1]
PASS: G1 computed distance 3 matches expected 3

___________________________________________________________________________
Testing G2...
Initializing optimizer and variables.
Generator matrix has 4 rows and 7 columns.
Added constraint: At least one row must be selected.
Defined Hamming weight as the sum of codeword bits.
Set optimization objective to minimize Hamming weight.
Starting optimization to find minimum non-zero Hamming distance.
Optimizer check resul